In [13]:
import glob
import xarray as xr
import netCDF4
import h5netcdf
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
file_path='../Data/'
data_list_in=glob.glob(file_path+'ds_in*')
data_list_out=glob.glob(file_path+'ds_out*')
data_list_in.sort()
data_list_out.sort()
print(data_list_in)
print(data_list_out)

['../Data/ds_in.csv', '../Data/ds_inBering.csv', '../Data/ds_inBering30day10.csv', '../Data/ds_inBering30day20.csv', '../Data/ds_inBering30day23.csv', '../Data/ds_inBering30day26.csv', '../Data/ds_inBering30day29.csv', '../Data/ds_inBering30day31.csv', '../Data/ds_inBering30day41.csv', '../Data/ds_inBering30day46.csv', '../Data/ds_inBering30day48.csv', '../Data/ds_inBering30day51.csv', '../Data/ds_inBering30day60.csv', '../Data/ds_inEastSH30day05.csv', '../Data/ds_inEastSH30day10.csv', '../Data/ds_inSH.csv', '../Data/ds_inSH30day10.csv', '../Data/ds_inSH30day20.csv', '../Data/ds_inSHWeddell30day10.csv', '../Data/ds_inSHWeddell30day20.csv', '../Data/ds_inWeddell.csv', '../Data/ds_in_30LeftFranday05.csv', '../Data/ds_in_30LeftFranday10.csv', '../Data/ds_in_30LeftFranday20.csv', '../Data/ds_in_30LeftFranday25.csv', '../Data/ds_in_30LeftFranday27.csv', '../Data/ds_in_30LeftFranday28.csv', '../Data/ds_in_30LeftFranday60.csv', '../Data/ds_in_30RightFranday05.csv', '../Data/ds_in_30RightFrand

In [3]:
data_arrays_in = [np.loadtxt(file, delimiter=',') for file in data_list_in]
data_arrays_out = [np.loadtxt(file, delimiter=',') for file in data_list_out]

# Print the shapes of the arrays to verify
for i, array in enumerate(data_arrays_in):
    print(f"Shape of data_arrays_in[{i}]: {array.shape}")

for i, array in enumerate(data_arrays_out):
    print(f"Shape of data_arrays_out[{i}]: {array.shape}")

/glade/derecho/scratch/geraint/tmp/ipykernel_9844/1827004139.py:1: UserWarning: loadtxt: input contained no data: "../Data/ds_inEastSH30day10.csv"
  data_arrays_in = [np.loadtxt(file, delimiter=',') for file in data_list_in]
/glade/derecho/scratch/geraint/tmp/ipykernel_9844/1827004139.py:2: UserWarning: loadtxt: input contained no data: "../Data/ds_outEastSH30day10.csv"
  data_arrays_out = [np.loadtxt(file, delimiter=',') for file in data_list_out]


Shape of data_arrays_in[0]: (45693, 13)
Shape of data_arrays_in[1]: (41227, 13)
Shape of data_arrays_in[2]: (30775, 13)
Shape of data_arrays_in[3]: (56466, 13)
Shape of data_arrays_in[4]: (95697, 13)
Shape of data_arrays_in[5]: (35113, 13)
Shape of data_arrays_in[6]: (31230, 13)
Shape of data_arrays_in[7]: (6052, 13)
Shape of data_arrays_in[8]: (17587, 13)
Shape of data_arrays_in[9]: (99267, 13)
Shape of data_arrays_in[10]: (19625, 13)
Shape of data_arrays_in[11]: (18723, 13)
Shape of data_arrays_in[12]: (123974, 13)
Shape of data_arrays_in[13]: (2029, 13)
Shape of data_arrays_in[14]: (0,)
Shape of data_arrays_in[15]: (44050, 13)
Shape of data_arrays_in[16]: (41869, 13)
Shape of data_arrays_in[17]: (3, 13)
Shape of data_arrays_in[18]: (16273, 13)
Shape of data_arrays_in[19]: (371, 13)
Shape of data_arrays_in[20]: (15356, 13)
Shape of data_arrays_in[21]: (50594, 13)
Shape of data_arrays_in[22]: (68024, 13)
Shape of data_arrays_in[23]: (19329, 13)
Shape of data_arrays_in[24]: (71206, 13)

In [4]:
# Reshape each array in data_arrays_in to the specified shape and concatenate them
reshaped_data_arrays_in = np.concatenate([array.reshape(-1, 13) for array in data_arrays_in])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_in: {reshaped_data_arrays_in.shape}")

Shape of reshaped_data_arrays_in: (1727878, 13)


In [5]:
# Reshape each array in data_arrays_out to the specified shape and concatenate them
reshaped_data_arrays_out = np.concatenate([array.reshape(-1, 12) for array in data_arrays_out])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_out: {reshaped_data_arrays_out.shape}")

Shape of reshaped_data_arrays_out: (1727878, 12)


In [6]:
# Clean data_arrays_in and data_arrays_out
cleaned_data_arrays_in = []
cleaned_data_arrays_out = []

for i in range(reshaped_data_arrays_in.shape[0]):
        if not np.sum(reshaped_data_arrays_out[i])==0:
            cleaned_data_arrays_in.append(reshaped_data_arrays_in[i])
            cleaned_data_arrays_out.append(reshaped_data_arrays_out[i])

# Convert lists back to numpy arrays if needed
cleaned_data_arrays_in = np.array(cleaned_data_arrays_in)
cleaned_data_arrays_out = np.array(cleaned_data_arrays_out)

# Print the shapes of the cleaned arrays to verify
print(f"Shape of cleaned_data_arrays_in: {cleaned_data_arrays_in.shape}")
print(f"Shape of cleaned_data_arrays_out: {cleaned_data_arrays_out.shape}")

Shape of cleaned_data_arrays_in: (1688479, 13)
Shape of cleaned_data_arrays_out: (1688479, 12)


In [18]:
# Check if cleaned_data_arrays_out has any zero values
has_zero_values = np.any(cleaned_data_arrays_out == 0)

print(f"Does cleaned_data_arrays_out have any zero values? {has_zero_values}")

# Find the indices of zero entries in cleaned_data_arrays_out
zero_indices = np.argwhere(cleaned_data_arrays_out == 0)

print(f"Indices of zero entries in cleaned_data_arrays_out: {zero_indices}")

Does cleaned_data_arrays_out have any zero values? True
Indices of zero entries in cleaned_data_arrays_out: [[   3226       0]
 [   3227       0]
 [   3228       0]
 ...
 [1688360       0]
 [1688360       1]
 [1688360       2]]


In [24]:
min_non_zero_value = np.min(np.abs(cleaned_data_arrays_out[np.nonzero(cleaned_data_arrays_out)]))
print(min_non_zero_value)
# So we add 1e-15 to all the zero entries in cleaned_data_arrays_out
cleaned_data_arrays_out += 1e-15

3.637978807091713e-12


In [26]:
cleaned_data_arrays_in_log=np.log10(np.abs(cleaned_data_arrays_in))
cleaned_data_arrays_out_sign=np.sign(cleaned_data_arrays_out)
cleaned_data_arrays_out_log=np.abs(np.log10(np.abs(cleaned_data_arrays_out)))
cleaned_data_arrays_out_log=cleaned_data_arrays_out_log*cleaned_data_arrays_out_sign

In [33]:
print(cleaned_data_arrays_out_log[1000000])
print(cleaned_data_arrays_out[1000000])

[ 5.30044061  4.82307707  4.48790337  4.21597172  3.98030505  3.77030576
  3.58116525  3.41049965  3.25705517  3.12014217  2.99936583 -2.47473637]
[ 5.00679016e-06  1.50287524e-05  3.25159635e-05  6.08174596e-05
  1.04639330e-04  1.69704843e-04  2.62322021e-04  3.88597808e-04
  5.53279824e-04  7.58329290e-04  1.00146129e-03 -3.35168839e-03]


In [35]:
#Make cleaned_data_arrays_out_log a tenth of the size
cleaned_data_arrays_out_log_10=cleaned_data_arrays_out_log[::10]
cleaned_data_arrays_in_log_10=cleaned_data_arrays_out_log[::10]
cleaned_data_arrays_out_log_10.shape

(168848, 12)

In [43]:
# Calculate the orders of magnitude difference
def calculate_orders_of_magnitude_difference(predictions, actuals):
    differences = torch.abs(predictions - actuals)
    orders_of_magnitude_diff = torch.log10(differences + 1e-10) - torch.log10(torch.abs(actuals) + 1e-10)
    return torch.mean(torch.abs(orders_of_magnitude_diff))

In [64]:
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.optim as optim

# Convert data to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in_log_10, cleaned_data_arrays_out_log_10, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network architecture
class PhysicsInformedNN(nn.Module):
    def __init__(self):
        super(PhysicsInformedNN, self).__init__()
        self.hidden1 = nn.Linear(X_train.shape[1], 64)
        self.hidden2 = nn.Linear(64, 64)
        self.hidden3 = nn.Linear(64, 64)
        self.output = nn.Linear(64, y_train_tensor.shape[1])
    
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

# Initialize the model, loss function, and optimizer
model = PhysicsInformedNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom loss function to enforce the sum of outputs to be zero
def custom_loss(output, target):
    mse_loss = criterion(output, target)

    sign_penalty = 0
    for i in range(output.shape[1]):
        sign_penalty += torch.mean(torch.relu(-output[:, i] * torch.sign(target[:, i])))
    sign_penalty = sign_penalty / output.shape[1]
    
    sign_loss=sign_penalty**2
    sum_constraint = torch.sum(output, dim=1)
    sum_loss = torch.mean(sum_constraint ** 2)
    return [mse_loss, sum_loss,sign_loss]

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = custom_loss(outputs, y_train_tensor)
    loss_sum= loss[0]+0.01*loss[1]#+500*loss[2] 
    loss_sum.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], MSE Loss: {loss[0].item():.6f} Sum loss: {loss[1].item():.6f} Sign Loss: {loss[2].item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = criterion(predictions, y_test_tensor)
    print(f'Test Mean Squared Error: {mse.item():.20f}')

Epoch [100/1000], MSE Loss: 3.722499 Sum loss: 2.293175 Sign Loss: 0.001909
Epoch [200/1000], MSE Loss: 3.271248 Sum loss: 2.108033 Sign Loss: 0.000200
Epoch [300/1000], MSE Loss: 3.122973 Sum loss: 2.110924 Sign Loss: 0.000053


KeyboardInterrupt: 

In [61]:
ds_in=cleaned_data_arrays_in_log_10
ds_out=cleaned_data_arrays_out_log_10

In [65]:
k=25009
print(model(torch.tensor(ds_in[k], dtype=torch.float32)))
print(sum(model(torch.tensor(ds_in[k], dtype=torch.float32))))
print(ds_in[k])
print(ds_out[k])

tensor([ 6.6293,  6.5585,  6.1153,  5.9348,  5.5327,  5.2596, -6.8299, -6.8221,
        -6.8502, -6.1782, -6.3806, -3.5757], grad_fn=<ViewBackward0>)
tensor(-0.6064, grad_fn=<AddBackward0>)
[ 6.06335189  5.56910131  5.21399603  4.9261397   4.67679143  4.4546046
 -7.57243077 -7.63459993 -7.47145762 -7.32506601 -7.19600317 -4.11080974]
[ 6.06335189  5.56910131  5.21399603  4.9261397   4.67679143  4.4546046
 -7.57243077 -7.63459993 -7.47145762 -7.32506601 -7.19600317 -4.11080974]


In [66]:
negative_sign_differences = 0
#ds_in_combine, ds_out_combine
for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    actual_output = torch.tensor(ds_out[i])
    
    # Compare the signs of the predicted and actual outputs
    predicted_signs = torch.sign(predicted_output)
    actual_signs = torch.sign(actual_output)
    
    # Count the number of different signs
    negative_sign_differences += torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of incorrect different negative signs: {negative_sign_differences} As a %: {100*negative_sign_differences/(len(ds_in)*len(ds_out[0]))}')

Total number of incorrect different negative signs: 79179 As a %: 3.907804652705392


In [67]:
# Calculate the mean absolute error (MAE) across all samples
def calculate_mean_absolute_error(predictions, actuals):
    absolute_errors = torch.abs(predictions - actuals)
    mean_absolute_error = torch.mean(absolute_errors)
    return mean_absolute_error

# Evaluate the model on the test set and calculate MAE
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    mae = calculate_mean_absolute_error(test_predictions, y_test_tensor)
    print(f'Test Mean Absolute Error: {mae.item():.6f}')

Test Mean Absolute Error: 1.336944


In [42]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    avg_orders_of_magnitude_diff = calculate_orders_of_magnitude_difference(test_predictions, y_test_tensor)
    print(f'Average Orders of Magnitude Difference: {avg_orders_of_magnitude_diff.item():.6f}')

NameError: name 'calculate_orders_of_magnitude_difference' is not defined

In [68]:
# Calculate the average of each model run
averages = 0

for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    average_output = torch.sum(predicted_output).item()
    averages+=average_output

# Print the averages
averages=averages/(len(ds_in)*len(ds_out[0]))
print(f"Average output for model run {i}: {averages:.20f}")

Average output for model run 168847: 0.06786338880884898450


In [59]:
predicted_output = model(torch.tensor(ds_in[2505], dtype=torch.float32))
average_output = torch.sum(predicted_output).item()
average_output

13.764627456665039

In [84]:
import pandas as pd

# Extract weights from the model
hidden1_weights = model.hidden1.weight.detach().numpy()
hidden2_weights = model.hidden2.weight.detach().numpy()
hidden3_weights = model.hidden3.weight.detach().numpy()

# Create a DataFrame for each layer's weights
df_hidden1 = pd.DataFrame(hidden1_weights)
df_hidden2 = pd.DataFrame(hidden2_weights)
df_hidden3 = pd.DataFrame(hidden3_weights)

# Save each DataFrame to a CSV file
df_hidden1.to_csv('hidden1_weights.csv', index=False, header=False)
df_hidden2.to_csv('hidden2_weights.csv', index=False, header=False)
df_hidden3.to_csv('hidden3_weights.csv', index=False, header=False)

print("Model weights saved to CSV files.")

Model weights saved to CSV files.


In [40]:
#Let's just try to classify sign
cleaned_data_arrays_out_sign=[]
for i in range(len(cleaned_data_arrays_out)):
    cleaned_data_arrays_out_sign.append(np.sign(cleaned_data_arrays_out[i]))
cleaned_data_arrays_out_sign=np.array(cleaned_data_arrays_out_sign)
print(cleaned_data_arrays_out_sign.shape)
print(cleaned_data_arrays_in.shape)

(131674, 12)
(131674, 13)


In [51]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in, cleaned_data_arrays_out_sign, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use `long` for integer labels
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(X_test.shape[1], 256)  # First layer with 64 units
        self.fc2 = nn.Linear(256, 128)  # Second layer with 32 units
        self.fc3 = nn.Linear(128, 64)  # Output layer
        self.fc4 = nn.Linear(64, y_test.shape[1],)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

# Define input and output sizes based on the tensor shapes
input_size = 13  # X has 13 features
output_size = 12  # y has 12 labels

# Instantiate the model, define loss and optimizer
model = BinaryClassifier(input_size, output_size)
criterion = nn.BCELoss()  # Binary cross-entropy for multi-label binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert y_train_tensor to range [0, 1] (optional)
y_train_tensor = (y_train_tensor + 1) / 2

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Convert outputs back to -1 or 1 after prediction if needed
final_outputs = (outputs * 2) - 1


X_train_tensor shape: torch.Size([105339, 13])
y_train_tensor shape: torch.Size([105339, 12])
Epoch [1/10], Loss: 0.6973
Epoch [2/10], Loss: 0.6917
Epoch [3/10], Loss: 0.6860
Epoch [4/10], Loss: 0.6798
Epoch [5/10], Loss: 0.6731
Epoch [6/10], Loss: 0.6658
Epoch [7/10], Loss: 0.6580
Epoch [8/10], Loss: 0.6491
Epoch [9/10], Loss: 0.6394
Epoch [10/10], Loss: 0.6285


In [ ]:
ds_in=cleaned_data_arrays_in
ds_out=cleaned_data_arrays_out_sign

In [52]:
negative_sign_differences = 0
#ds_in_combine, ds_out_combine
for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    actual_output = torch.tensor(ds_out[i])
    
    # Compare the signs of the predicted and actual outputs
    predicted_signs = torch.sign(predicted_output)
    actual_signs = torch.sign(actual_output)
    
    # Count the number of different signs
    negative_sign_differences += torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of different negative signs: {negative_sign_differences} As a %: {100*negative_sign_differences/(len(ds_in)*len(ds_in[0]))}')

Total number of different negative signs: 382523 As a %: 22.346739792097267
